Run:

In [1]:
import re, requests, os

The webpage version of some bcy.net's posts are broken, which means the http get method will not work here.
Now let us try a different, half-machinic method.

### Use Fiddler 4
Fiddler is a proxy that scans ongoing web traffic through my local machines. I can set up such a monitor on my desktop which is connected to the same network (wifi) my Android phones connects to in order to grab the page source.
The bcy.net Android app can fetch the posts in the json format. We monitor this app.

Openning Fiddler 4, I will do some settings. Go to `tools -> Options` menu. Under `HTTPS`, check these:
  * Capture HTTPS CONNECTs
  * Decrypt HTTPS traffic
  * Ignore server certification errors
 
After enabling them, go to `Connections`, and check "Allow remote computers to connect". Remember the port number: it is `8888` by default.

#### Get dekstop's IP address
Hit powershell, and type `ipconfig`. Remember the "IPv4" address.

### Settings on Android
On my phone, I will need to connect to the same network. Beyond that, go to the hotspot-specific settings, under 'proxy', fill out the computer IP and port.

Now go to the browser and type `IP address:port` on the address bar, for example, `192.168.0.9:8888`. Follow the page and install certificate quickly.

The phone side is then ready-to-go. Open a random webpage and the app should display the connection.

Now switch to the desktop and do some filter work: I only monitor connections to my bcy connections of interest. Go to Filters tab the the right side in Fiddler, check `Use filters`, and fill the domain under `Show only the following Hosts`. The domain is `api.bcy.net`.

Also, every post url has the following format: 
`api.bcy.net/apiv2/item/detail?iid=......`
So under the "Request headers" section, in the "Show only if URL contains" box, fill in "item/detail".
By previous observation, such pages are in json format.

And now open the bcy app, tap a post, and a new session will appear. I can do this repeatedly. I do not need to wait for the full loading of each post.


### Export and process
Under `File -> Export sessions -> all sessions` then select `row files`. Open that dumped folder, I will see many json files with the filenames of session numberings. Open one of them I can see its content: it is indeed the full post containing image URLs. I now need to extract these urls.

The following code combines all the text into one file:

Run this under the dump folder.

In [ ]:
with open('res.txt', 'w', encoding='utf-8') as res:
	for file in os.listdir('.'):
		with open(file, encoding='utf-8') as js:
			res.write(js.read())
		res.write('\n\n')

Do an analysis.
I will see each image url is in the following form:

`https://p1-bcy.byteimg.com/img/banciyuan/coser/10927/post/178ym/4cbe8950872311e7aa4699045f6ccdbe.jpg~tplv-banciyuan-w650.image`

So this regex filters urls:

`r'https://p\d-bcy\.byteimg\.com.+?image'`

Also note some urls are avatars which we do not want. Hence we have


In [ ]:
with open('res.txt', encoding='utf-8') as file:
    v = re.findall(r'https://p\d-bcy\.byteimg\.com.+?image', file.read())
    v = [i for i in v if 'avatar' not in i]

Run this. This defines a list of image urls. Note, however, that this url is just a thumbnail. Luckily, its real path is not difficult to find (after I observed some samples). Take a look at this demo:

In [2]:
thumbPath = 'https://p1-bcy.byteimg.com/img/banciyuan/coser/10927/post/178ym/4cbe8950872311e7aa4699045f6ccdbe.jpg~tplv-banciyuan-w650.image'
realPath = re.sub(r'.+?banciyuan/', 'https://img-bcy-qn.pstatp.com/', thumbPath)
realPath = re.sub(r'~.+', '', realPath)
realPath

'https://img-bcy-qn.pstatp.com/coser/10927/post/178ym/4cbe8950872311e7aa4699045f6ccdbe.jpg'

We can then conclude the code in a downloader:

In [ ]:
count = 0
for each in v:
    reg = re.search(r'.+/(.+?)/(.+?)~.+?', each).groups()
    foldername = reg[0]
    filename = reg[1]
    os.makedirs(foldername, exist_ok=True)
    if os.path.exists(os.path.join(foldername, filename)):
        count += 1
        continue

    try:
        # modify url
        each = re.sub(r'.+?banciyuan/', 'https://img-bcy-qn.pstatp.com/', each)
        each = re.sub(r'~.+', '', each)
        # modify url end
        
        req = requests.get(each)
        with open(os.path.join(foldername, filename), 'wb') as file:
            for chunk in req.iter_content(1000):
                file.write(chunk)
    except Exception as e:
        print(str(e))
    count += 1
    print(count)

Run this and start downloading